# Bonus: on-line smile detection

In this lab, you will write a classifier that recognises automatically whether the people behind behind the computer are smiling or not. This is a challenging classification problem, and requires us to deal with: 
1. handling the camera hardware
1. Detecting the face(s) in the image
1. Deciding whether the face contains a smile.

To help you implement this, the following code reads in the images from the webcam, detects any faces, resizes the detected faces to a cannonical size of 40 by 40 pixels and calls the classification function (which you need to implement) to decide whether that face sports a smile or not.

In [1]:
# First, let's import the libraries that we need

import cv2
import numpy as np
import sklearn as sk

face_target_size = (48,48)


## Smile detection

In the lecture, we have seen different types of classifiers. We have looked at naive probabilistic classifiers, which consider each dimension independently, and we have seen more advanced classifiers (such as linear classifiers and Decision Trees) which consider the relations between the different dimensions in the data. For the bonus, you can use existing implementations of classifiers and the challenge is to figure out what kind of classifier is appropriate for this problem.

In [2]:

def hasSmile(face):
    # Your implementation comes here
    return False

## Training

You are given the following labelled data to train your classifier on. It consists of 7519 images of smiles, and 5221 images of other faces. The following code reads them into two matrices, where each row contains the pixels of one image; it is up to you to convert them to a format that your classifier understands.


In [3]:
from os import listdir

posdir = 'smiles/pos'
negdir = 'smiles/neg'

pos = listdir(posdir)
neg = listdir(negdir)
print(len(pos),len(neg))

posfaces = np.zeros((len(pos),face_target_size[0]*face_target_size[1]))

Visualise = False
if Visualise:
    print("Press 'q' to stop the iteration!")
    print("Set 'Visualise=False' to speed things up.")

i=0
for p in pos:
    im = cv2.imread(posdir+'/'+p, cv2.IMREAD_GRAYSCALE)
    if Visualise:
        print(p, im.shape)
        cv2.imshow("face",im)
    posfaces[i,:] = im.flatten()
    i += 1
    
    if Visualise:
        k = cv2.waitKey(500) 
        if k!=-1:
            print(k)
        if k & 0xFF == ord('q'):
            break
if Visualise:    
    cv2.destroyWindow("face")

negfaces = np.zeros((len(neg),face_target_size[0]*face_target_size[1]))

i=0
for p in neg:
    im = cv2.imread(negdir+'/'+p, cv2.IMREAD_GRAYSCALE)
    negfaces[i,:] = im.flatten()
    i+=1

print(posfaces.shape,negfaces.shape)
    

7519 5221
(7519, 2304) (5221, 2304)


## Evaluation

To evaluate your implementation, we will run it on a separate test set of face images. This set is safely kept in secret conditions and you are not allowed to see it or use it to improve your classifier. This is to avoid overfitting: you cannot tweak your classifier to perform well on the particular data that we will evaluate you on. 

But you should be able to evaluate your work as well, before you submit it. This requires you to have labelled data too. You should therefore split the data you have loaded in, into a train and a validation set. The performance on the train set indicates how well your classifier is learning from that data; the performance on the validation set gives you an indication of how well the classifier will perform on future data.

## Visualisation

You can additionally evaluate your method on yourself with the following code. This code uses the first webcam it detects and shows the captured image. The detected faces are shown with a box, and this box becomes red if your code detects a smile. Enjoy!

In [4]:
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model

face_detection = load_detection_model("./haarcascade_frontalface_default.xml")

emotion_offsets = (20, 40)

cv2.namedWindow('webcam')
cap = cv2.VideoCapture(0)
while True:
    ret,bgr_image = cap.read()
    if not ret:
        break
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    faces = detect_faces(face_detection, gray_image)        

    for face_coordinates in faces:
        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        if x2<=x1 or y2 <= y1:
            continue
        gray_face = gray_image[y1:y2, x1:x2]
        gray_face = cv2.resize(gray_face, face_target_size)

        color = (255,255,255)
        if hasSmile(gray_face): 
            print(x1,x2,y1,y2)
            color = (255,0,0)
        x,y,w,h = face_coordinates

        draw_bounding_box(face_coordinates, rgb_image, color)
#        cv2.rectangle(rgb_image, (x,y-5),(x+w,y-15), color, 1)
    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

    cv2.imshow('webcam',bgr_image)
    k = cv2.waitKey(10) 
    if k!=-1:
        print(k)
    if k & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyWindow("webcam")

Using TensorFlow backend.


113
